In [2]:
import pandas as pd
import datetime
import pickle

cars_df = pd.read_csv("LoanTap_Data_Streamlit.csv")

C:\Users\itviv\AppData\Local\Temp\ipykernel_16064\897637726.py:5: DtypeWarning: Columns (2,4,5,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  cars_df = pd.read_csv("LoanTap_Data_Streamlit.csv")


In [3]:
cars_df.head()

,loan_amnt,installment,grade,annual_inc,loan_status,purpose,dti,revol_util,application_type,address
0,10000.0,329.48,B,117000.0,Fully Paid,vacation,26.24,41.8,INDIVIDUAL,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,265.68,B,65000.0,Fully Paid,debt_consolidation,22.05,53.3,INDIVIDUAL,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,506.97,B,43057.0,Fully Paid,credit_card,12.79,92.2,INDIVIDUAL,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,220.65,A,54000.0,Fully Paid,credit_card,2.60,21.5,INDIVIDUAL,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,609.33,C,55000.0,Charged Off,credit_card,33.95,69.8,INDIVIDUAL,"679 Luna Roads\r\nGreggshire, VA 11650"


In [8]:
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# 2. Features and target and Train-test split
data = cars_df.dropna()

loan_label= {'Fully Paid':0, 'Charged Off':1}
data['loan_status'] = data['loan_status'].replace(loan_label)
data['zip_code'] = data['address'].astype(str).str[-5:]

X = data[data.columns.drop(['loan_status','address'])]
y = data['loan_status']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

cat_cols = ['grade','zip_code','application_type','purpose']

te = TargetEncoder(cols=cat_cols)
X_train[cat_cols] = te.fit_transform(X_train[cat_cols], y_train)
X_test[cat_cols] = te.transform(X_test[cat_cols])

C:\Users\itviv\AppData\Local\Temp\ipykernel_16064\2675221527.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['loan_status'] = data['loan_status'].replace(loan_label)
C:\Users\itviv\AppData\Local\Temp\ipykernel_16064\2675221527.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['loan_status'] = data['loan_status'].replace(loan_label)
C:\Users\itviv\AppData\Local\Temp\ipykernel_16064\2675221527.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [9]:
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=10,
    eval_metric='logloss',
    random_state=42
)

# Fit model
xgb_model.fit(X_train, y_train)

# 6. Predict on test set
y_pred = xgb_model.predict(X_test)

# 7. Evaluate
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"Test mape: {mape}")

Test mape: 44728417983791.336
